In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

## Read an image of a the target chessboard in base position
im2 = cv.imread("C:/Users/sbsch/Downloads/ChessBoard(3).png")

## This section reads each square of the board and save the countours of each unique piece in a list 
## to check future contours against and determine piece type
target_squares = [[1,0,"Pawn"],[0,1,"Knight"],[0,2,"Bishop"],[0,0,"Rook"],[0,3,"Queen"],[0,4,"King"]]
control_contours = []

for square in target_squares:
    i = square[1]
    j = square[0]
    piece = square[2]
    
    x = 123*j
    y = 123*i
    # x = 99*j
    # y = 99*i

    square = im2[x:x+123 , y:y+123]

    # Convert the image to grayscaleq
    gray = cv.cvtColor(square, cv.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce image noise and improve contour detection
    blurred = cv.GaussianBlur(gray, (3, 3), 0)

    # Use adaptive thresholding to create a binary image
    thresh = cv.adaptiveThreshold(blurred, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 7, 1)

    # Find contours in the thresholded image
    contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    temp_contours = []

    min_contour_area = 2000  # Minimum contour area to be considered a piece
    max_contour_area = 10000

    for cnt in contours:
        if min_contour_area <  cv.contourArea(cnt) < max_contour_area:
            temp_contours.append(cnt)
            control_contours.append(cnt)
            #np.save(f'{piece}.npy', temp_contours)
            
        else:
            pass
            
    

In [ ]:
# ### Testing the Control Contours

# im2 = cv.imread("C:/Users/sbsch/Downloads/ChessBoard(3).png")
# #loaded_contours = np.load('Bishop.npy', allow_pickle=True)

# cv.drawContours(im2, control_contours[4], -1, (0, 255, 0), 3)
# cv.imshow("Detected Pieces", im2)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [ ]:
import cv2 as cv
from PIL import Image, ImageGrab 
import numpy as np
import matplotlib.pyplot as plt

letters = ['a','b','c','d','e','f','g','h']

#cv.imshow("ChessBoard", im2)

# pawn_contour = np.load('Pawn.npy', allow_pickle=True)
# knight_contour = np.load('Knight.npy', allow_pickle=True)
# bishop_contour = np.load('Bishop.npy', allow_pickle=True)
# rook_contour = np.load('Rook.npy', allow_pickle=True)
# queen_contour = np.load('Queen.npy', allow_pickle=True)
# king_contour = np.load('King.npy', allow_pickle=True)
#pieces_contours = [pawn_contour, knight_contour, bishop_contour, rook_contour, queen_contour, king_contour]


def shift_contour(contour, x_offset, y_offset):
    # Create a copy of the contour to avoid modifying the original
    shifted_contour = contour.copy()
    # Add the offset to all points in the contour
    shifted_contour[:,:,1] += x_offset  # Shift X coordinates
    shifted_contour[:,:,0] += y_offset  # Shift Y coordinates
    return shifted_contour

def piece_identifier(contour):
    similarity_array = []
    for piece in control_contours:
        similarity = cv.matchShapes(piece, contour, cv.CONTOURS_MATCH_I1, 0.0)
        similarity_array.append(similarity)
    #print(similarity_array)
    if similarity_array.index(min(similarity_array)) == 0:
        return "p" #"Pawn"
    elif similarity_array.index(min(similarity_array)) == 1:
        return "n" #"Knight"
    elif similarity_array.index(min(similarity_array)) == 2:
        return "b" #"Bishop"
    elif similarity_array.index(min(similarity_array)) == 3:
        return "r" #"Rook"
    elif similarity_array.index(min(similarity_array)) == 4:
        return "q" #"Queen"
    elif similarity_array.index(min(similarity_array)) == 5:
        return "k" #"King"
    else:
        pass


def board_finder(board_im):
    im_rgb = cv.cvtColor(board_im, cv.COLOR_BGR2RGB)

    # Define the color to match (in RGB)
    target_color = [237, 214, 176]

    # Find all pixels matching the target color
    mask = cv.inRange(im_rgb, np.array(target_color), np.array(target_color))
    matching_pixels = cv.findNonZero(mask)

    xArray = []
    yArray = []
    if matching_pixels is not None:
        
        
        for pixel in matching_pixels:
            x, y = pixel[0]
            xArray.append(x)
            xArray = list(set(xArray))
            yArray.append(y)
            yArray = list(set(yArray))
            
            cv.circle(im_rgb, (x, y), 5, (0, 255, 0), -1)

        left = min(xArray)
        right = max(xArray) + 1
        top = min(yArray)
        bottom = max(yArray) + 1

    plt.imshow(im_rgb)
    plt.show()
    print(left, right, top, bottom)
    #237, 214, 176
    return left, right, top, bottom

def board_state(board_img_file,square_size):
    
    im2 = cv.imread(board_img_file)    
    black_pieces_contours = [ ]
    white_pieces_contours = [ ]
    board = [["." for _ in range(8)] for _ in range(8)]
    
    for i in range(8):
        for j in range(8):

            # You can add an offset to the contours to shift them into the appropriate position of the board
            x = square_size*i
            y = square_size*j

            square = im2[x:x+square_size, y:y+square_size]

            # Convert the image to grayscaleq
            gray = cv.cvtColor(square, cv.COLOR_BGR2GRAY)

            # Apply GaussianBlur to reduce image noise and improve contour detection
            blurred = cv.GaussianBlur(gray, (3, 3), 0)

            # Use adaptive thresholding to create a binary image
            thresh = cv.adaptiveThreshold(blurred, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 7, 1)

            # Find contours in the thresholded image
            contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
            
            temp_contours = []
            
            min_contour_area = 2000  # Minimum contour area to be considered a piece
            max_contour_area = 10000
            
            for cnt in contours:
                if min_contour_area <  cv.contourArea(cnt) < max_contour_area:
                    temp_contours.append(cnt)
                    piece_color = square[100, 64]
                    piece_type = piece_identifier(temp_contours[0])
                    #print(piece_color)
                else:
                    pass
                
            if len(temp_contours) > 0:
                if piece_color[0] == 87:
                    shifted_contour = shift_contour(temp_contours[0], x, y)
                    black_pieces_contours.append(shifted_contour)
                    #print(f"Piece: Black {piece_type} | Square: {letters[j]}{8-i}")
                    board[i][j] = f"{piece_type}"
                elif piece_color[0] == 249:
                    shifted_contour = shift_contour(temp_contours[0], x, y)
                    white_pieces_contours.append(shifted_contour)
                    #print(f"Piece: White {piece_type} | Square: {letters[j]}{8-i}")
                    board[i][j] = f"{piece_type.capitalize()}"
                else:
                    pass
            else:
                board[i][j] = "."
    # for row in board:
    #     print(row)
    #     print()
    # Draw contours on the original image
    # im2_with_contours = im2.copy()
    # cv.drawContours(im2_with_contours, black_pieces_contours, -1, (255, 0, 0), 2)
    # cv.drawContours(im2_with_contours, white_pieces_contours, -1, (0, 255, 0), 2)

    # # Display the image with detected pieces
    # cv.imshow("Detected Pieces", im2_with_contours)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    return board


In [ ]:
from PIL import Image, ImageGrab
# top = 255 #170
# left = 223 #411
# right = left + board_size#792
# bottom = top + board_size#792

ImageGrab.grab().save("testScreenshot.png")

testim = cv.imread("testScreenshot.png")
left, right, top, bottom = board_finder(testim)
board_size = right - left
square_size = int(board_size/8)

ImageGrab.grab(bbox = (left, top, right, bottom)).save("beforeBoard.png")
im = cv.imread("beforeBoard.png")
# cv.imshow("BeforeBoard",im)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [ ]:
from PIL import Image, ImageGrab
import os
import time
import csv
import keyboard
# top = 255
# left = 223
# right = left + board_size
# bottom = top + board_size

moveList = []

def capture_board():
    ImageGrab.grab(bbox = (left, top, right, bottom)).save("afterBoard.png")

    board1 = board_state("beforeBoard.png",square_size)
    #print("\n")
    board2 = board_state("afterBoard.png",square_size)

    start_square = ""
    end_square  = ""

    for i in range(8):
        for j in range(8):
            if board1[i][j] != board2[i][j]:
                
                if board2[i][j] == ".":
                    start_square = f"{letters[j]}{8-i}"
                else:
                    end_square = f"{letters[j]}{8-i}"
            
            else:

                pass
    
    if start_square != "" and end_square != "":
        print(f"{start_square}{end_square}")
        moveList.append(start_square+end_square)
    else:
        pass

    os.remove("beforeBoard.png")
    os.rename("afterBoard.png", "beforeBoard.png")
    return

while True:
    capture_board()
    time.sleep(0.2)
    with open('moveList.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        for move in moveList:
            writer.writerow([move])
    if keyboard.is_pressed('q'): 
        print("Exiting loop.")
        break

#capture_board()

